In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
import matplotlib.pyplot as plt
from collections import defaultdict
from datetime import datetime
plt.rcParams['font.family'] = 'SimHei' 

In [9]:
#ticker_train_data
ticker_train_data_raw = pd.read_json("ticker_train_data.json")
ticker_train_data_raw = ticker_train_data_raw.set_index('Date')
expanded_data1 = ticker_train_data_raw['Affected Companies'].apply(pd.Series)
ticker_train_data = ticker_train_data_raw.drop(columns=['Affected Companies']).join(expanded_data1)

#ticker_test_data
ticker_test_data_raw = pd.read_json("ticker_test_data.json")
ticker_test_data_raw = ticker_test_data_raw.set_index('Date')
expanded_data2 = ticker_test_data_raw['Affected Companies'].apply(pd.Series)
ticker_test_data = ticker_test_data_raw.drop(columns=['Affected Companies']).join(expanded_data2)

In [10]:
# 检查
print(ticker_train_data)
print(ticker_test_data)

                 CVX      AAPL       WMT       WBA        GS       MCD  \
Date                                                                     
2020-09-01  negative  positive  positive  negative   neutral  negative   
2020-09-02       NaN  negative  positive       NaN  positive       NaN   
2020-09-03  negative  negative       NaN       NaN  positive  negative   
2020-09-04       NaN  negative  negative       NaN  positive  negative   
2020-09-08       NaN  negative       NaN       NaN  negative   neutral   
...              ...       ...       ...       ...       ...       ...   
2021-09-24  negative  negative       NaN       NaN   neutral       NaN   
2021-09-27   neutral  negative       NaN       NaN  positive   neutral   
2021-09-28       NaN       NaN  positive       NaN  negative  positive   
2021-09-29   neutral   neutral       NaN       NaN   neutral       NaN   
2021-09-30       NaN   neutral       NaN  positive  negative       NaN   

                  VZ        KO      A

In [11]:
#stock_list
stock_list = [
    'MMM', 'AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'DOW', 'HON',
    'INTC', 'IBM', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'CRM', 'KO',
    'GS', 'HD', 'PG', 'TRV', 'DIS', 'UNH', 'VZ', 'V', 'WBA', 'WMT'
]

#stock_market_data
stock_data_df = pd.read_excel("DOW30_2020-2024.xlsx")
stock_data_df = stock_data_df.rename(columns={
    'Names Date': 'Date',
    'Ticker Symbol': 'Ticker',
    'Bid or Low Price': 'Low',
    'Ask or High Price': 'High',
    'Price or Bid/Ask Average': 'Close',
    'Open Price': 'Open',
    'Volume': 'Volume'
})
stock_data_df['Date'] = pd.to_datetime(stock_data_df['Date'])

#时间范围（后续会修改）
start_date = '2020-09-01'
end_date = '2022-12-30'
filtered_data = stock_data_df[(stock_data_df['Date'] >= start_date) & (stock_data_df['Date'] <= end_date)]
filtered_data = filtered_data[['Date', 'Ticker', 'Open', 'Close', 'High', 'Low', 'Volume']]

# 检查
print(filtered_data)

#stock_market_data字典
stock_market_data = {}
for _, row in filtered_data.iterrows():
    ticker = row['Ticker']
    date_str = row['Date'].strftime('%Y-%m-%d') 
    open_price = float(row['Open'])
    close_price = float(row['Close']) 
    high_price = float(row['High'])
    low_price = float(row['Low'])
    volume = float(row['Volume'])
    price_data = [open_price, close_price, high_price, low_price, volume]

    if ticker not in stock_market_data:
        stock_market_data[ticker] = {}

    stock_market_data[ticker][date_str] = price_data

# 检查
print(stock_market_data)

#检查一下
print("检查stock data...")
print(f"Number of stocks: {len(stock_market_data)}")
print(f"Average trading days per stock: {sum(len(dates) for dates in stock_market_data.values()) / len(stock_market_data)}")
missing_tickers = [ticker for ticker in stock_list if ticker not in stock_market_data]
if missing_tickers:
    print(f"Missing tickers: {missing_tickers}")
print(f"Stock tickers: {stock_market_data.keys()}")

            Date Ticker        Open       Close        High         Low  \
168   2020-09-01   MSFT  225.509995  227.270004  227.449997  224.429993   
169   2020-09-02   MSFT  227.970001  231.649994  232.860001  227.350006   
170   2020-09-03   MSFT  229.270004  217.300003  229.309998  214.960205   
171   2020-09-04   MSFT  215.100006  214.250000  218.359894  205.190002   
172   2020-09-08   MSFT  206.500000  202.660004  210.029999  202.199997   
...          ...    ...         ...         ...         ...         ...   
37233 2022-12-23    UNH  524.099976  531.309998  531.309998  522.895020   
37234 2022-12-27    UNH  533.929993  531.989990  535.840027  529.844971   
37235 2022-12-28    UNH  535.070007  528.450012  538.150024  527.734009   
37236 2022-12-29    UNH  532.539978  529.880005  533.679993  528.859985   
37237 2022-12-30    UNH  530.000000  530.179993  530.500000  524.840027   

         Volume  
168    25729137  
169    34009970  
170    58217130  
171    59426793  
172    52

In [12]:
#得到股票价格数据的训练集和测试集的日期
train_dates = sorted(ticker_train_data.index.unique().astype(str))
test_dates = sorted(ticker_test_data.index.unique().astype(str))

#把股票价格数据分成训练集和测试集
train_stock_data = {}
test_stock_data = {}
for ticker, date_data in stock_market_data.items():
    train_stock_data[ticker] = {}
    test_stock_data[ticker] = {}
    for date, price_data in date_data.items():
        if date in train_dates:
            train_stock_data[ticker][date] = price_data
        elif date in test_dates:
            test_stock_data[ticker][date] = price_data

print(f"Training date range: {train_dates[0]} to {train_dates[-1]}, total {len(train_dates)} days")
print(f"Test date range: {test_dates[0]} to {test_dates[-1]}, total {len(test_dates)} days")

Training date range: 2020-09-01 to 2021-09-30, total 273 days
Test date range: 2021-10-01 to 2022-12-30, total 315 days


In [13]:
print(train_stock_data)

{'MSFT': {'2020-09-01': [225.509994506836, 227.270004272461, 227.449996948242, 224.429992675781, 25729137.0], '2020-09-02': [227.970001220703, 231.649993896484, 232.860000610352, 227.350006103516, 34009970.0], '2020-09-03': [229.270004272461, 217.300003051758, 229.309997558594, 214.960205078125, 58217130.0], '2020-09-04': [215.100006103516, 214.25, 218.359893798828, 205.190002441406, 59426793.0], '2020-09-08': [206.5, 202.660003662109, 210.029998779297, 202.199996948242, 52667527.0], '2020-09-09': [207.600006103516, 211.289993286133, 214.839904785156, 206.699996948242, 45547702.0], '2020-09-10': [213.399993896484, 205.369995117187, 214.740005493164, 204.110000610352, 35386842.0], '2020-09-11': [207.199996948242, 204.029998779297, 208.630004882812, 201.240005493164, 33753495.0], '2020-09-14': [204.240005493164, 205.410003662109, 209.199996948242, 204.029998779297, 30285989.0], '2020-09-15': [208.419998168945, 208.779998779297, 209.779998779297, 206.929992675781, 21794854.0], '2020-09-16

In [ ]:
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
    
    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

class LSTM_Model(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(LSTM_Model, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
    
    def forward(self, x):
        output, (hidden, _) = self.lstm(x)
        return output[:, -1, :] 

class StockPredictionModel(nn.Module):
    def __init__(self, company_feature_dim, stock_feature_dim, hidden_dim, output_dim):
        super(StockPredictionModel, self).__init__()
        self.gnn = GNN(company_feature_dim, hidden_dim)#GNN处理公司特征和邻居信息
        self.lstm_company = LSTM_Model(hidden_dim + stock_feature_dim, hidden_dim)#GNN的输出+股票价格信息
        self.lstm_stock = LSTM_Model(stock_feature_dim, hidden_dim)#只有股票价格信息
        self.fc = nn.Linear(hidden_dim * 2, output_dim)#最后用MLP处理GNN和LSTM的输出
    
    def forward(self, gnn_x, edge_index, company_seq, stock_seq):
        gnn_output = self.gnn(gnn_x, edge_index) #h_i,t^GNN
        #LSTM处理公司特征和邻居信息
        company_lstm_output = self.lstm_company(company_seq)  # h_i,t^COMB
        #LSTM处理股票价格信息
        stock_lstm_output = self.lstm_stock(stock_seq)  # h_i,t^STOCK
        #合并
        combined = torch.cat((company_lstm_output, stock_lstm_output), dim=1)
        output = self.fc(combined)  # y_i = MLP(CONCAT(h_i,t^COMB, h_i,t^STOCK); θ_MLP)
        
        return F.softmax(output, dim=1)

print("Loading and processing data...")

# Build graphs for training data
print("Building graphs for training data...")
train_graphs = {}
companies_features = {}

# Get all unique stock tickers
all_tickers = set()
for date, row in ticker_train_data.iterrows():
    for col in row.index:
        if col not in ['Date'] and isinstance(row[col], str) and row[col] in ['positive', 'neutral', 'negative']:
            all_tickers.add(col)

# Build a graph for each date
for date, row in ticker_train_data.iterrows():
    date_str = date.strftime('%Y-%m-%d') if isinstance(date, pd.Timestamp) else date
    G = nx.Graph()
    affected_companies = {}
    for col in row.index:
        if col not in ['Date']:
            ticker = col
            sentiment = row[col]
            if isinstance(sentiment, str) and sentiment in ['positive', 'neutral', 'negative']:
                sentiment_value = 1 if sentiment == 'positive' else (-1 if sentiment == 'negative' else 0)
                affected_companies[ticker] = sentiment_value
                if ticker not in companies_features:
                    companies_features[ticker] = {}
                companies_features[ticker][date_str] = sentiment_value
    
    # Add nodes with sentiment values
    for ticker, sentiment_value in affected_companies.items():
        G.add_node(ticker, sentiment=sentiment_value)
    
    # Add edges between companies mentioned on the same day
    tickers = list(affected_companies.keys())
    for ticker1, ticker2 in combinations(tickers, 2):
        G.add_edge(ticker1, ticker2)
    
    train_graphs[date_str] = G

# Build graphs for test data
print("Building graphs for test data...")
test_graphs = {}

for date, row in ticker_test_data.iterrows():
    date_str = date.strftime('%Y-%m-%d') if isinstance(date, pd.Timestamp) else date
    G = nx.Graph()
    affected_companies = {}
    for col in row.index:
        if col not in ['Date']:
            ticker = col
            sentiment = row[col]
            if isinstance(sentiment, str) and sentiment in ['positive', 'neutral', 'negative']:
                sentiment_value = 1 if sentiment == 'positive' else (-1 if sentiment == 'negative' else 0)
                affected_companies[ticker] = sentiment_value
    
    # Add nodes with sentiment values
    for ticker, sentiment_value in affected_companies.items():
        G.add_node(ticker, sentiment=sentiment_value)
    
    # Add edges between companies mentioned on the same day
    tickers = list(affected_companies.keys())
    for ticker1, ticker2 in combinations(tickers, 2):
        G.add_edge(ticker1, ticker2)
    
    test_graphs[date_str] = G

#这里可以调参，10d效果不好
lookback_window = 3

# For each stock, prepare training and validation data
print(f"Preparing training and validation data for {len(stock_list)} stocks...")
models = {}
performance = {}

# # 如果在这里直接定义model呢？
# hidden_dim = 64
# model = StockPredictionModel(
#     company_feature_dim=1, 
#     stock_feature_dim=5,  
#     hidden_dim=hidden_dim,
#     output_dim=3)
    
# # Define loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=1e-5)

for ticker in stock_list:

    # 如果在这里直接定义model呢？
    hidden_dim = 64
    model = StockPredictionModel(
    company_feature_dim=1, 
    stock_feature_dim=5,  
    hidden_dim=hidden_dim,
    output_dim=3)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=1e-5)

    print(f"\nProcessing stock: {ticker}")
    
    # Prepare training samples
    train_samples = []
    
    # For each possible window
    for i in range(len(train_dates) - lookback_window - 1):
        # Get window dates and target date
        window_dates = train_dates[i:i+lookback_window]  # Features
        target_date = train_dates[i+lookback_window]  # Label
        
        # 1. Prepare GNN input features
        gnn_features = []
        edge_indices = []
        
        for date_idx, date in enumerate(window_dates):
            graph = train_graphs.get(date, nx.Graph())
            
            # Get node features
            sentiment = 0  # Default neutral
            if ticker in graph.nodes:
                sentiment = graph.nodes[ticker].get('sentiment', 0)
            
            gnn_features.append([sentiment])
            
            # Collect edge indices
            if ticker in graph.nodes:
                edge_indices.append([date_idx, date_idx])  # Self-loop edge
                for neighbor in graph.neighbors(ticker):
                    edge_indices.append([date_idx, date_idx])  # Connection edge
        
        # 2. Prepare LSTM sequences
        # company_seq = []
        # stock_seq = []
        
        # for date in window_dates:
        #     gnn_output = np.zeros(64)  # Assume 64-dimensional GNN output
        #     stock_data = np.zeros(5)  # Default all zeros
        #     if ticker in train_stock_data and date in train_stock_data[ticker]:
        #         stock_data = train_stock_data[ticker][date]
        #     # Concatenate GNN output and stock data
        #     combined_feat = np.concatenate([gnn_output, stock_data])
        #     company_seq.append(combined_feat)
        #     stock_seq.append(stock_data)

        # 如果在这里直接定义model呢？
        # hidden_dim = 64
        # model = StockPredictionModel(
        # company_feature_dim=1, 
        # stock_feature_dim=5,  
        # hidden_dim=hidden_dim,
        # output_dim=3)

        company_seq = []
        stock_seq = []

        # 确保 model 已经初始化，并且 .gnn 部分可用
        model.gnn.eval()
        with torch.no_grad():
            for date in window_dates:
                # 1) 构造当日 PyG 输入
                G = train_graphs.get(date, nx.Graph())
                # stock_list: 全局 DOW30 按固定顺序的列表
                # 这里的x_list是30家公司当天股票涨跌预测（GPT给出的）
                x_list = [[ G.nodes.get(n,{}).get('sentiment', 0) ] for n in stock_list]
                # print(x_list)
                # print(len(x_list))
                # breakpoint()
                x = torch.FloatTensor(x_list)  # shape [N, 1]
                # print(x)
                edges = list(G.edges())
                # print(edges)
                if not edges:
                    edge_index = torch.LongTensor([[0],[0]])
                else:
                    idx_map = {n:i for i,n in enumerate(stock_list)}
                    src = [idx_map[u] for u,v in edges] + [idx_map[v] for u,v in edges]
                    dst = [idx_map[v] for u,v in edges] + [idx_map[u] for u,v in edges]
                    edge_index = torch.LongTensor([src, dst])  # [2, 2*E]

                # 2) 真正跑一次 GNN
                # 将当天所有公司的涨跌预测（GPT给出的）和边（GPT给出的）喂给GNN
                h = model.gnn(x, edge_index)  # tensor [N, hidden_dim]

                # 3) 取出当前 ticker 的 embedding
                idx = stock_list.index(ticker)
                h_i = h[idx].cpu().numpy()    # (hidden_dim,)

                # 4) 读取当日行情
                stock_feature_dim_temp = 5
                stock_data = train_stock_data.get(ticker,{}).get(date, np.zeros(stock_feature_dim_temp))

                # 5) 拼接
                # 循环后得到的 company_seq 和 stock_seq 中包含的是一个窗口内的输出
                company_seq.append( np.concatenate([h_i, stock_data]) )
                stock_seq.append(stock_data)
        
        # 3. Prepare label
        # Calculate actual price change to determine label
        if (ticker in train_stock_data and 
            target_date in train_stock_data[ticker] and 
            train_dates[i+lookback_window-1] in train_stock_data[ticker]):
            
            price_t = train_stock_data[ticker][target_date][1]  # Target day closing price
            price_t_1 = train_stock_data[ticker][train_dates[i+lookback_window-1]][1]  # Previous day closing price
            
            if price_t_1 > 0:
                return_rate = price_t / price_t_1 - 1
                
                # 用市场数据给股票每天的涨跌打标签
                # Determine label based on return rate threshold
                if return_rate >= 0.01:  # Up (1% threshold)
                    label = 0  # Up
                elif return_rate <= -0.01:  # Down
                    label = 2  # Down
                else:
                    label = 1  # Flat
            else:
                label = 1  # Default flat
        else:
            label = 1  # Default flat
        
        # If valid data exists, add sample
        if len(gnn_features) > 0 and len(company_seq) > 0 and len(stock_seq) > 0:
            train_samples.append({
                'gnn_features': torch.FloatTensor(gnn_features),
                'edge_index': torch.LongTensor(edge_indices).t().contiguous() if edge_indices else torch.LongTensor([[0], [0]]),
                'company_seq': torch.FloatTensor([company_seq]),
                'stock_seq': torch.FloatTensor([stock_seq]),
                'label': label
            })

    # print(train_samples[label])
    
    if len(train_samples) == 0:
        print(f"  No training samples, skipping {ticker}")
        models[ticker] = None
        continue
    
    # Training-validation split
    # 这里使用的 train_data 和 val_data 中 gnn_features 和 edge_index 是GPT处理后获得数据，company_seq 和 stock_seq 是GNN处理后获得数据
    # label 是原始市场数据处理后获得的标签
    train_size = int(len(train_samples) * 0.8)
    train_data = train_samples[:train_size]
    val_data = train_samples[train_size:]
    
    print(f"  Training samples: {len(train_data)}, Validation samples: {len(val_data)}")
    
    # Train model
    print(f"  Starting model training...")
    
    # Initialize model
    # hidden_dim = 64
    # model = StockPredictionModel(
    #     company_feature_dim=1, 
    #     stock_feature_dim=5,  
    #     hidden_dim=hidden_dim,
    #     output_dim=3     
    # )
    
    # Define loss function and optimizer
    # criterion = nn.CrossEntropyLoss()
    # optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=1e-5)
    
    # Training parameters
    best_val_loss = float('inf')
    no_improve_epochs = 0
    best_model = None
    train_losses = []
    val_losses = []
    patience = 10
    num_epochs = 50 # 循环的回合数
    batch_size = 16
    
    # Training loop
    for epoch in range(num_epochs):
        # 这里直接使用train方法而不用给参数吗？
        model.train()
        total_train_loss = 0
        
        # 打乱测试数据
        np.random.shuffle(train_data)
        
        # 分批训练，这里batch_size为步长
        for i in range(0, len(train_data), batch_size):
            batch = train_data[i:i+batch_size]
            batch_loss = 0
            
            for sample in batch:
                # Forward pass
                outputs = model(
                    sample['gnn_features'],
                    sample['edge_index'],
                    sample['company_seq'],
                    sample['stock_seq']
                )
                
                # 计算损失
                loss = criterion(outputs, torch.tensor([sample['label']]))
                batch_loss += loss
            
            # 反向传播与优化
            optimizer.zero_grad()
            batch_loss.backward()
            # 梯度裁剪，防止爆炸
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            total_train_loss += batch_loss.item()
        
        # 计算平均训练损失
        avg_train_loss = total_train_loss / (len(train_data) / batch_size)
        train_losses.append(avg_train_loss)
        
        # 验证
        model.eval()
        total_val_loss = 0
        correct = 0
        
        with torch.no_grad():
            for sample in val_data:
                outputs = model(
                    sample['gnn_features'],
                    sample['edge_index'],
                    sample['company_seq'],
                    sample['stock_seq']
                )
                
                loss = criterion(outputs, torch.tensor([sample['label']]))
                total_val_loss += loss.item()
                
                # 计算准确度
                pred = outputs.argmax(dim=1).item()
                correct += (pred == sample['label'])
        
        # 计算平均验证损失和准确度
        avg_val_loss = total_val_loss / len(val_data)
        val_accuracy = correct / len(val_data)
        val_losses.append(avg_val_loss)
        
        # 打印训练过程
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.4f}')

        # print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.4f}')


    # 对每个ticker依次保存model
    # Save model
    models[ticker] = model
    
    # Prepare test samples
    test_samples = []
    

    # 在测试集上进行验证
    # For each possible window
    for i in range(len(test_dates) - lookback_window - 1):
        # Get window dates and target date
        window_dates = test_dates[i:i+lookback_window]  # Features
        target_date = test_dates[i+lookback_window]  # Label
        
        # 1. Prepare GNN input features
        gnn_features = []
        edge_indices = []
        
        for date_idx, date in enumerate(window_dates):
            graph = test_graphs.get(date, nx.Graph())
            
            # Get node features
            sentiment = 0  # Default neutral
            if ticker in graph.nodes:
                sentiment = graph.nodes[ticker].get('sentiment', 0)
            
            gnn_features.append([sentiment])
            
            # Collect edge indices
            if ticker in graph.nodes:
                edge_indices.append([date_idx, date_idx])  # Self-loop edge
                for neighbor in graph.neighbors(ticker):
                    edge_indices.append([date_idx, date_idx])  # Connection edge
        
        # # 2. Prepare LSTM sequences
        # company_seq = []
        # stock_seq = []
        
        # for date in window_dates:
        #     gnn_output = np.zeros(64)  # Assume 64-dimensional GNN output
        #     stock_data = np.zeros(5)  # Default all zeros
        #     if ticker in test_stock_data and date in test_stock_data[ticker]:
        #         stock_data = test_stock_data[ticker][date]
        #     # Concatenate GNN output and stock data
        #     combined_feat = np.concatenate([gnn_output, stock_data])
        #     company_seq.append(combined_feat)
        #     stock_seq.append(stock_data)

        company_seq = []
        stock_seq = []

        # 确保 model 已经初始化，并且 .gnn 部分可用
        model.gnn.eval()
        with torch.no_grad():
            for date in window_dates:
                # 1) 构造当日 PyG 输入
                G = train_graphs.get(date, nx.Graph())
                # stock_list: 全局 DOW30 按固定顺序的列表
                # 这里的x_list是30家公司当天股票涨跌预测（GPT给出的）
                x_list = [[ G.nodes.get(n,{}).get('sentiment', 0) ] for n in stock_list]
                # print(x_list)
                # print(len(x_list))
                # breakpoint()
                x = torch.FloatTensor(x_list)  # shape [N, 1]
                # print(x)
                edges = list(G.edges())
                # print(edges)
                if not edges:
                    edge_index = torch.LongTensor([[0],[0]])
                else:
                    idx_map = {n:i for i,n in enumerate(stock_list)}
                    src = [idx_map[u] for u,v in edges] + [idx_map[v] for u,v in edges]
                    dst = [idx_map[v] for u,v in edges] + [idx_map[u] for u,v in edges]
                    edge_index = torch.LongTensor([src, dst])  # [2, 2*E]

                # 2) 真正跑一次 GNN
                # 将当天所有公司的涨跌预测（GPT给出的）和边（GPT给出的）喂给GNN
                h = model.gnn(x, edge_index)  # tensor [N, hidden_dim]

                # 3) 取出当前 ticker 的 embedding
                idx = stock_list.index(ticker)
                h_i = h[idx].cpu().numpy()    # (hidden_dim,)

                # 4) 读取当日行情
                stock_feature_dim_temp = 5
                stock_data = train_stock_data.get(ticker,{}).get(date, np.zeros(stock_feature_dim_temp))

                # 5) 拼接
                # 循环后得到的 company_seq 和 stock_seq 中包含的是一个窗口内的输出
                company_seq.append( np.concatenate([h_i, stock_data]) )
                stock_seq.append(stock_data)
        
        # 3. 准备标签
        # 计算实际价格变化来判断标签
        if (ticker in test_stock_data and 
            target_date in test_stock_data[ticker] and 
            test_dates[i+lookback_window-1] in test_stock_data[ticker]):
            
            price_t = test_stock_data[ticker][target_date][1]  # Target day closing price
            price_t_1 = test_stock_data[ticker][test_dates[i+lookback_window-1]][1]  # Previous day closing price
            
            if price_t_1 > 0:
                return_rate = price_t / price_t_1 - 1
                
                # Determine label based on return rate threshold
                if return_rate >= 0.01:  # Up (1% threshold)
                    label = 0  # Up
                elif return_rate <= -0.01:  # Down
                    label = 2  # Down
                else:
                    label = 1  # Flat
            else:
                label = 1  # Default flat
        else:
            label = 1  # Default flat
        
        # If valid data exists, add sample
        if len(gnn_features) > 0 and len(company_seq) > 0 and len(stock_seq) > 0:
            test_samples.append({
                'gnn_features': torch.FloatTensor(gnn_features),
                'edge_index': torch.LongTensor(edge_indices).t().contiguous() if edge_indices else torch.LongTensor([[0], [0]]),
                'company_seq': torch.FloatTensor([company_seq]),
                'stock_seq': torch.FloatTensor([stock_seq]),
                'label': label
            })
    
    if len(test_samples) > 0:
        # Evaluate model
        model.eval()
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for sample in test_samples:
                # outputs 为三维数据，给出三种分类的概率分布
                outputs = model(
                    sample['gnn_features'],
                    sample['edge_index'],
                    sample['company_seq'],
                    sample['stock_seq']
                )
                
                # 选取预测概率最大的值作为预测值
                pred = outputs.argmax(dim=1).item()
                all_preds.append(pred)
                all_labels.append(sample['label'])

            print(all_preds)

        weighted_f1 = f1_score(all_labels, all_preds, average='weighted')
        macro_f1 = f1_score(all_labels, all_preds, average='macro')
        micro_f1 = f1_score(all_labels, all_preds, average='micro')
        
        # 分类报告
        report = classification_report(all_labels, all_preds, target_names=['Up', 'Flat', 'Down'])
        
        eval_results = {
            'weighted_f1': weighted_f1,
            'macro_f1': macro_f1,
            'micro_f1': micro_f1,
            'predictions': all_preds,
            'labels': all_labels,
            'report': report
        }
        
        performance[ticker] = eval_results
        
        print(f"  Test results - Weighted F1 score: {eval_results['weighted_f1']:.4f}")
        print(f"  Classification report:")
        print(eval_results['report'])
    else:
        print(f"  No test samples, skipping evaluation for {ticker}")

# Build and evaluate portfolio
print("\nBuilding and evaluating portfolio...")

# 在测试集上进行策略回测

# Initialize portfolio
initial_capital = 1000000
cash = initial_capital
positions = defaultdict(float)  # Holdings
position_values = defaultdict(float)  # Position values
portfolio_values = [initial_capital]
dates = [test_dates[lookback_window]]

# Record days held for each stock
holding_days = defaultdict(int)

# Configuration parameters
MAX_HOLDING_DAYS = 5  # Maximum holding days (one week)
CASH_RESERVE = 0.3  # Keep 30% cash reserve
POSITION_LIMIT = 0.2  # Maximum position ratio for a single stock
BUY_CONFIDENCE = 0.65  # Buy confidence threshold
SELL_CONFIDENCE = 0.6  # Sell confidence threshold

# For each trading day
for i in range(lookback_window, len(test_dates) - 1):
    current_date = test_dates[i]
    next_date = test_dates[i + 1]
    
    # Update holding days for all positions
    for ticker in positions:
        if positions[ticker] > 0:
            holding_days[ticker] += 1
    
    # Calculate current portfolio total value (cash + positions)
    total_value = cash
    for ticker, shares in positions.items():
        if ticker in test_stock_data and current_date in test_stock_data[ticker]:
            current_price = test_stock_data[ticker][current_date][1]
            position_values[ticker] = shares * current_price
            total_value += position_values[ticker]
    
    # Forced liquidation strategy - stocks held for too long
    for ticker in list(positions.keys()):
        if positions[ticker] > 0 and holding_days[ticker] >= MAX_HOLDING_DAYS:
            if ticker in test_stock_data and current_date in test_stock_data[ticker]:
                current_price = test_stock_data[ticker][current_date][1]
                cash += positions[ticker] * current_price
                print(f"Date {current_date}: Forced liquidation {ticker}, held for {holding_days[ticker]} days, gained cash {positions[ticker] * current_price:.2f}")
                positions[ticker] = 0
                position_values[ticker] = 0
                holding_days[ticker] = 0
    
    # Get all stocks predicted to rise
    up_predictions = {}
    down_predictions = {}
    
    for ticker, model in models.items():
        if model is None:
            continue
        model.eval()
        
        # Extract data from the past lookback_window days
        window_dates = test_dates[i-lookback_window:i]
        
        # Prepare model inputs
        gnn_features = []
        edge_indices = []
        company_seq = []
        stock_seq = []
        
        # Get sentiment features
        for date_idx, date in enumerate(window_dates):
            graph = test_graphs.get(date, nx.Graph())
            
            sentiment = 0  # Default neutral
            if ticker in graph.nodes:
                sentiment = graph.nodes[ticker].get('sentiment', 0)
            
            gnn_features.append([sentiment])
            
            # Collect edge indices (simplified)
            if ticker in graph.nodes:
                edge_indices.append([date_idx, date_idx])
        
        # Get sequence data
        for date in window_dates:
            gnn_output = np.zeros(64)  # 占位符
            
            stock_data = np.zeros(5)
            if ticker in test_stock_data and date in test_stock_data[ticker]:
                stock_data = test_stock_data[ticker][date]
            
            combined_feat = np.concatenate([gnn_output, stock_data])
            company_seq.append(combined_feat)
            stock_seq.append(stock_data)
        
        # If valid data exists, make prediction
        if len(gnn_features) > 0 and len(company_seq) > 0:
            with torch.no_grad():
                outputs = model(
                    torch.FloatTensor(gnn_features),
                    torch.LongTensor(edge_indices).t().contiguous() if edge_indices else torch.LongTensor([[0], [0]]),
                    torch.FloatTensor([company_seq]),
                    torch.FloatTensor([stock_seq])
                )
                
                pred = outputs.argmax(dim=1).item()
                probs = outputs.numpy()[0]  # Prediction probabilities
                
                # Current price
                if ticker in test_stock_data and current_date in test_stock_data[ticker]:
                    current_price = test_stock_data[ticker][current_date][1]
                    
                    # Collect predictions
                    if pred == 0 and probs[0] > BUY_CONFIDENCE:  # Predicted rise
                        up_predictions[ticker] = (probs[0], current_price)
                    elif pred == 2 and probs[2] > SELL_CONFIDENCE:  # Predicted fall
                        down_predictions[ticker] = (probs[2], current_price)
    
    # First execute sell operations - sell stocks predicted to fall
    for ticker, (confidence, price) in down_predictions.items():
        if positions[ticker] > 0:
            cash += positions[ticker] * price
            print(f"Date {current_date}: Sell {ticker}, confidence {confidence:.2f}, gained cash {positions[ticker] * price:.2f}")
            positions[ticker] = 0
            position_values[ticker] = 0
            holding_days[ticker] = 0
    
    # Update portfolio total value
    total_value = cash
    for ticker, shares in positions.items():
        if ticker in test_stock_data and current_date in test_stock_data[ticker]:
            current_price = test_stock_data[ticker][current_date][1]
            position_values[ticker] = shares * current_price
            total_value += position_values[ticker]
    
    # Calculate funds available for new investments (reserve a certain percentage of cash)
    available_cash = max(0, cash - total_value * CASH_RESERVE)
    
    # Buy logic - allocate funds to stocks predicted to rise
    if up_predictions and available_cash > 0:
        # Sort by confidence
        sorted_predictions = sorted(up_predictions.items(), key=lambda x: x[1][0], reverse=True)
        
        # Calculate allocatable funds per stock
        allocation_per_stock = min(
            available_cash / len(sorted_predictions),
            total_value * POSITION_LIMIT  # Limit single stock position
        )
        
        for ticker, (confidence, price) in sorted_predictions:
            # Check if already holding
            existing_value = position_values[ticker]
            
            # Calculate buyable amount (considering existing position)
            max_position_value = total_value * POSITION_LIMIT
            additional_allocation = max(0, min(allocation_per_stock, max_position_value - existing_value))
            
            if additional_allocation > 0 and price > 0:
                shares_to_buy = additional_allocation / price
                positions[ticker] += shares_to_buy
                cash -= shares_to_buy * price
                holding_days[ticker] = max(holding_days[ticker], 1)  # Update or initialize holding days
                
                print(f"Date {current_date}: Buy {ticker}, confidence {confidence:.2f}, invested {shares_to_buy * price:.2f}")
                
                # Update available funds
                available_cash -= shares_to_buy * price
                if available_cash <= 0:
                    break
    
    # Calculate portfolio value at the end of the day
    portfolio_value = cash
    for ticker, shares in positions.items():
        if ticker in test_stock_data and current_date in test_stock_data[ticker]:
            current_price = test_stock_data[ticker][current_date][1]
            portfolio_value += shares * current_price
    
    portfolio_values.append(portfolio_value)
    dates.append(next_date)

# Calculate portfolio metrics
returns = [portfolio_values[i] / portfolio_values[i-1] - 1 for i in range(1, len(portfolio_values))]
cumulative_returns = [portfolio_values[i] / initial_capital - 1 for i in range(len(portfolio_values))]

# Volatility (annualized)
volatility = np.std(returns) * np.sqrt(252)  # Assume 252 trading days per year

# Sharpe ratio (simplified, assume risk-free rate is 0)
sharpe_ratio = (np.mean(returns) * 252) / volatility if volatility > 0 else 0

# Maximum drawdown
peak = portfolio_values[0]
max_drawdown = 0

for value in portfolio_values:
    if value > peak:
        peak = value
    drawdown = (peak - value) / peak
    max_drawdown = max(max_drawdown, drawdown)

portfolio_results = {
    'portfolio_values': portfolio_values,
    'dates': dates,
    'returns': returns,
    'cumulative_returns': cumulative_returns,
    'total_return': portfolio_values[-1] / initial_capital - 1,
    'volatility': volatility,
    'sharpe_ratio': sharpe_ratio,
    'max_drawdown': max_drawdown
}

# Output portfolio performance
print("\nPortfolio Performance:")
print(f"Total return: {portfolio_results['total_return'] * 100:.2f}%")
print(f"Sharpe ratio: {portfolio_results['sharpe_ratio']:.4f}")
print(f"Volatility: {portfolio_results['volatility'] * 100:.2f}%")
print(f"Maximum drawdown: {portfolio_results['max_drawdown'] * 100:.2f}%")

# Plot portfolio performance
plt.figure(figsize=(10, 6))
plt.plot(portfolio_results['dates'], portfolio_results['cumulative_returns'], 'b-', linewidth=2)
plt.grid(True, linestyle='--', alpha=0.7)
plt.title('Portfolio Cumulative Returns', fontsize=14)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cumulative Returns', fontsize=12)
plt.tight_layout()
plt.savefig('portfolio_returns.png')
plt.close()

print("Results chart saved as 'portfolio_returns.png'")

Loading and processing data...
Building graphs for training data...
Building graphs for test data...
Preparing training and validation data for 30 stocks...

Processing stock: MMM
  Training samples: 215, Validation samples: 54
  Starting model training...
Epoch [10/50], Training Loss: 15.8826, Validation Loss: 0.7922, Accuracy: 0.7593
Epoch [20/50], Training Loss: 15.2211, Validation Loss: 0.7922, Accuracy: 0.7593
Epoch [30/50], Training Loss: 15.2231, Validation Loss: 0.7922, Accuracy: 0.7593
Epoch [40/50], Training Loss: 15.2231, Validation Loss: 0.7922, Accuracy: 0.7593
Epoch [50/50], Training Loss: 15.2231, Validation Loss: 0.7922, Accuracy: 0.7593
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

d:\Apps\Anaconda\conda\envs\ml_proj\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Apps\Anaconda\conda\envs\ml_proj\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Apps\Anaconda\conda\envs\ml_proj\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


  Training samples: 215, Validation samples: 54
  Starting model training...
Epoch [10/50], Training Loss: 18.8455, Validation Loss: 1.0513, Accuracy: 0.5000
Epoch [20/50], Training Loss: 18.0675, Validation Loss: 1.0514, Accuracy: 0.5000
Epoch [30/50], Training Loss: 17.9022, Validation Loss: 1.0514, Accuracy: 0.5000
Epoch [40/50], Training Loss: 17.9022, Validation Loss: 1.0514, Accuracy: 0.5000
Epoch [50/50], Training Loss: 17.9022, Validation Loss: 1.0514, Accuracy: 0.5000
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

d:\Apps\Anaconda\conda\envs\ml_proj\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Apps\Anaconda\conda\envs\ml_proj\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Apps\Anaconda\conda\envs\ml_proj\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


  Training samples: 215, Validation samples: 54
  Starting model training...
Epoch [10/50], Training Loss: 18.0512, Validation Loss: 0.8355, Accuracy: 0.7222
Epoch [20/50], Training Loss: 15.2975, Validation Loss: 0.8292, Accuracy: 0.7222
Epoch [30/50], Training Loss: 15.2975, Validation Loss: 0.8292, Accuracy: 0.7222
Epoch [40/50], Training Loss: 15.2975, Validation Loss: 0.8292, Accuracy: 0.7222
Epoch [50/50], Training Loss: 15.2975, Validation Loss: 0.8292, Accuracy: 0.7222
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

...

  Training samples: 215, Validation samples: 54
  Starting model training...
Epoch [10/50], Training Loss: 15.0683, Validation Loss: 0.7192, Accuracy: 0.8333
Epoch [20/50], Training Loss: 14.0324, Validation Loss: 0.7181, Accuracy: 0.8333
Epoch [30/50], Training Loss: 14.0324, Validation Loss: 0.7181, Accuracy: 0.8333
Epoch [40/50], Training Loss: 14.0324, Validation Loss: 0.7181, Accuracy: 0.8333
Epoch [50/50], Training Loss: 14.0324, Validation Loss: 0.7181, Accuracy: 0.8333
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

d:\Apps\Anaconda\conda\envs\ml_proj\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Apps\Anaconda\conda\envs\ml_proj\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Apps\Anaconda\conda\envs\ml_proj\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Date 2021-10-12: Forced liquidation AAPL, held for 5 days, gained cash 199309.85
Date 2021-10-12: Buy AAPL, confidence 1.00, invested 199861.97
Date 2021-10-13: Buy AAPL, confidence 1.00, invested 677.92
Date 2021-10-18: Forced liquidation AAPL, held for 5 days, gained cash 207685.29
Date 2021-10-18: Buy AAPL, confidence 1.00, invested 201291.05
Date 2021-10-22: Forced liquidation AAPL, held for 5 days, gained cash 204230.41
Date 2021-10-22: Buy AAPL, confidence 1.00, invested 201878.92
Date 2021-10-25: Buy AAPL, confidence 1.00, invested 54.31
Date 2021-10-28: Forced liquidation AAPL, held for 5 days, gained cash 207202.62
Date 2021-10-28: Buy AAPL, confidence 1.00, invested 202932.80
Date 2021-10-29: Buy AAPL, confidence 1.00, invested 2947.50
Date 2021-11-01: Buy AAPL, confidence 1.00, invested 907.04
Date 2021-11-03: Forced liquidation AAPL, held for 5 days, gained cash 205399.49
Date 2021-11-03: Buy AAPL, confidence 1.00, invested 202655.23
Date 2021-11-04: Buy AAPL, confidence 1.

C:\Users\27583\AppData\Local\Temp\ipykernel_3608\1436986035.py:788: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) SimHei.
  plt.tight_layout()
C:\Users\27583\AppData\Local\Temp\ipykernel_3608\1436986035.py:789: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) SimHei.
  plt.savefig('portfolio_returns.png')


Results chart saved as 'portfolio_returns.png'
